In [1]:
import connectorx as cx
import pandas as pd
import numpy as np
import os
import duckdb
from datetime import datetime
import glob
import multiprocessing as mp
from functools import partial
import time

In [2]:
# 买方价格偏离度因子
bid_factor_lists = [
    'bid_pre_open_915_920',
    'bid_pre_open_920_925',
    'bid_early_930_1000',
    'bid_main_1000_1430',
    'bid_late_1430_1457',
    'bid_close_1457_1500'
]

# 卖方价格偏离度因子
ask_factor_lists = [
    'ask_pre_open_915_920',
    'ask_pre_open_920_925',
    'ask_early_930_1000',
    'ask_main_1000_1430',
    'ask_late_1430_1457',
    'ask_close_1457_1500'
]

all_factors = bid_factor_lists + ask_factor_lists

factor_name = "six_itvs_price_divergence"

factor_dir = "/data/home/lexuanchen/.conda/envs/Order_Improvement/factors/six_itvs_price_divergence"

In [4]:
parquet_pattern = os.path.join(factor_dir, "*.parquet")
parquet_files = glob.glob(parquet_pattern)
print(f"在目录 {factor_dir} 中找到 {len(parquet_files)} 个Parquet文件")


conn = duckdb.connect(database=':memory:')
all_factor_data = conn.execute(f"""
    SELECT * FROM read_parquet('{parquet_pattern}')
""").fetchdf()
conn.close()


all_factor_data['date'] = pd.to_datetime(all_factor_data['date'])

all_factor_data = all_factor_data.sort_values(['date', 'security_code'])

output_path = "/data/home/lexuanchen/Factors/Order/Signal/Raw_Six_Times_Divergence"
os.makedirs(output_path, exist_ok=True)

for factor in all_factors:

    factor_df = all_factor_data[['date','security_code', factor]]
    result_df = factor_df.dropna(subset=[factor])

    output_file_path = f"{output_path}/{factor}.csv"

    result_df.to_csv(output_file_path,index=False)


在目录 /data/home/lexuanchen/.conda/envs/Order_Improvement/factors/six_itvs_price_divergence 中找到 854 个Parquet文件


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))